In [1]:
import tensorflow as tf

下面首先定义一个图（其实没有必要，tensorflow会默认定义一个），并做一些计算。

In [2]:
graph = tf.Graph()
with graph.as_default():
    foo = tf.Variable(3, name='foo')
    bar = tf.Variable(2, name='bar')
    result = foo + bar
    initialize = tf.global_variables_initializer()

In [3]:
print(result)

Tensor("add:0", shape=(), dtype=int32)


这段代码，首先会载入tensorflow，定义一个graph类，并在这张图上定义了foo与bar的两个变量，最后对这个值求和，并初始化所有变量。其中，Variable是定义变量并赋予初值。让我们看下result（最后1行代码）。后面是输出，可以看到并没有输出实际的结果，由此可见在定义图的时候其实没有进行任何实际的计算。

下面定义一个session，并进行真正的计算

In [4]:
with tf.Session(graph=graph) as sess:
    sess.run(initialize)
    res = sess.run(result)
print(res)

5


这段代码中，定义了session，并在session中执行了真正的初始化，并且求得result的值并打印出来。可以看到，在session中产生了真正的计算，得出值为5。

下图是该graph在tensorboard中的显示。这张图整体是一个graph,其中foo,bar,add这些节点都是operation，而foo和bar与add连接边的就是tensor。当session运行result时，实际就是求得add这个operation流出的tensor值，那么add的所有上游节点都会进行计算，如果图中有非add上游节点（本例中没有）那么该节点将不会进行计算，这也是图计算的优势之一。

![](http://img.blog.csdn.net/20170403072111314?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvamRiYw==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/Center)

## 数据结构
Tensorflow的数据结构有着rank,shape,data types的概念，下面来分别讲解。

（1）rank

Rank一般是指数据的维度，其与线性代数中的rank不是一个概念。其常用rank举例如下。
![](http://img.blog.csdn.net/20170403072140955?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvamRiYw==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/Center)

(2) shape
shape指的是tensor每个维度数据的个数，可以用python的list/tuple表示。下图表示了rank, shape的关系
![](http://img.blog.csdn.net/20170403072215267?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvamRiYw==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/Center)

(3)data type
Data type,是指单个数据的类型。常用DT_FLOAT，也就是32位的浮点数。下图展示了所有的types
![](http://img.blog.csdn.net/20170403072300580?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvamRiYw==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/Center)



### Variabels
（1）介绍

当训练模型时，需要使用Variables保存与更新参数。Variables会保存在内存当中，所有tensor一旦拥有Variables的指向就不会在session中丢失。其必须明确的初始化而且可以通过Saver保存到磁盘上。Variables可以通过Variables初始化。

In [5]:
weights = tf.Variable(tf.random_normal([784, 200], stddev=0.35),name="weights")
biases = tf.Variable(tf.zeros([200]), name="biases")

其中，tf.random_normal是随机生成一个正态分布的tensor，其shape是第一个参数，stddev是其标准差。tf.zeros是生成一个全零的tensor。之后将这个tensor的值赋值给Variable。

（2）初始化

实际在其初始化过程中做了很多的操作，比如初始化空间，赋初值（等价于tf.assign），并把Variable添加到graph中等操作。注意在计算前需要初始化所有的Variable。一般会在定义graph时定义global_variables_initializer，其会在session运算时初始化所有变量。

直接调用global_variables_initializer会初始化所有的Variable，如果仅想初始化部分Variable可以调用tf.variables_initializer。

In [7]:
Init_ab = tf.variables_initializer([foo,bar],name='init_ab')

Variables可以通过eval显示其值，也可以通过assign进行赋值。Variables支持很多数学运算，具体可以参照官方文档。
（3）Variables与constant的区别

值得注意的是Variables与constant的区别。Constant一般是常量，可以被赋值给Variables，constant保存在graph中，如果graph重复载入那么constant也会重复载入，其非常浪费资源，如非必要尽量不使用其保存大量数据。而Variables在每个session中都是单独保存的，甚至可以单独存在一个参数服务器上。可以通过代码观察到constant实际是保存在graph中，具体如下。

In [8]:
const = tf.constant(1.0, name='constant')
print(tf.get_default_graph().as_graph_def())

node {
  name: "random_normal/shape"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
          dim {
            size: 2
          }
        }
        tensor_content: "\020\003\000\000\310\000\000\000"
      }
    }
  }
}
node {
  name: "random_normal/mean"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
        }
        float_val: 0.0
      }
    }
  }
}
node {
  name: "random_normal/stddev"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
        }
        float_val: 0.3499999940395355
      }
    }
  }
}
node {
  name: "random_normal/RandomStandardNormal"
  op: "RandomStandardNormal"
 

（4）命名

另外一个值得注意的地方是尽量每一个变量都明确的命名，这样易于管理命令空间，而且在导入模型的时候不会造成不同模型之间的命名冲突，这样就可以在一张graph中容纳很多个模型。

### placeholders与feed_dict

当我们定义一张graph时，有时候并不知道需要计算的值，比如模型的输入数据，其只有在训练与预测时才会有值。这时就需要placeholder与feed_dict的帮助。

定义一个placeholder，可以使用tf.placeholder(dtype,shape=None,name=None)函数。

In [9]:
foo = tf.placeholder(tf.int32, shape=[1], name='foo')
bar = tf.constant(2, name='bar')
result = foo + bar
with tf.Session() as sess:
    print(sess.run(result))

InvalidArgumentError: You must feed a value for placeholder tensor 'foo' with dtype int32 and shape [1]
	 [[Node: foo = Placeholder[dtype=DT_INT32, shape=[1], _device="/job:localhost/replica:0/task:0/gpu:0"]()]]
	 [[Node: add/_3 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_8_add", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'foo', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-

在上面的代码中，会抛出错误（InvalidArgumentError），因为计算result需要foo的具体值，而在代码中并没有给出。这时候需要将实际值赋给foo。最后一行修改如下（其中最后的dict就是一个feed_dict，一般会使用python读入一些值后传入，当使用minbatch的情况下，每次输入的值都不同）：

In [10]:
foo = tf.placeholder(tf.int32, shape=[1], name='foo')
bar = tf.constant(2, name='bar')
result = foo + bar
with tf.Session() as sess:
    print(sess.run(result, {foo: [3]}))

[5]


## mnist识别实例
介绍了一些tensorflow基础后，我们用一个完整的例子将这些串起来。

首先，需要下载数据集，mnist数据可以在[Yann LeCun's website](http://yann.lecun.com/exdb/mnist/)下载到，也可以通过如下两行代码得到。

In [11]:
from tensorflow.examples.tutorials.mnist import input_dataut_data

In [13]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


该数据集中一共有55000个样本，其中50000用于训练，5000用于验证。每个样本分为X与y两部分，其中X如下图所示，是28*28的图像，在使用时需要拉伸成784维的向量。

![](http://img.blog.csdn.net/20170403072635347?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvamRiYw==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/Center)

整体X可以表示成:
![](http://img.blog.csdn.net/20170403072721363?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvamRiYw==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/Center)

y为X真实的类别，其数据可以看做如下图的形式。因此，问题可以看成一个１０分类的问题
![](http://img.blog.csdn.net/20170403072757270?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvamRiYw==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/Center)

而本次演示所使用的模型都为逻辑回归，其可以表示成
![](http://img.blog.csdn.net/20170403072859080?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvamRiYw==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/Center)

使用tensorflow进行graph构建时，大体可以分为五部分
1. 为输入Ｘ与输出y定义placeholder
2. 定义权重W
3. 定义模型结构
4. 定义损失函数
5. 定义优化算法
首先导入需要的包，定义x与y的placeholder以及w,b的Variables。其中None表示任意维度，一般是mini-batch的batch size
其实W定义是shape为784,10 rank 定义２的Variable, b是shape为10,rank为１的Variable

In [18]:
import tensorflow as tf
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

之后是定义模型，x与W矩阵乘法后与b求和，经过softmax得到y

In [19]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

求逻辑回归的损失函数，这是使用了cross entropy，其公式可以表示成

![](http://img.blog.csdn.net/20170403073221319?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvamRiYw==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/Center)

这里的cross entropy取了均值。定义了学习步长为0.5，使用了梯度下降算法(GradientDescentOptimizer)最小化损失函数。不要忘记初始化Variables。

In [20]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
init = tf.global_variables_initializer()

最后，我们的 graph 至此定义完毕，下面就可以进行真正的计算，包括初始化变量，输入数据，并计算损失函数与利用优化算法更新参数。

In [25]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_:mnist.test.labels}))

0.9159001


其中，迭代了1000次，每次输入了100个样本。mnist.train.next_batch 就是生成下一个 batch 的数据，这里知道它在干什么就可以。那么训练结果如何呢，需要进行评估。这里使用单纯的正确率，正确率是用取最大值索引是否相等的方式，因为正确的 label 最大值为1，而预测的 label 最大值为最大概率。